## Import Libraries

In [1]:
#!/usr/bin/env python
# coding: utf-8
import os
import numpy as np
import pandas as pd
import requests
from datetime import datetime
import json

In [2]:
# Variables
season = "2020-21"

## Import files

In [3]:
all_events = pd.read_csv(r'./data/' + season + '/Cleaned_data/all_events.csv', parse_dates= ["deadline_time"])
all_players = pd.read_csv(r'./data/' + season + '/Merged_data/all_players_merged.csv', parse_dates= ["news_added"]) 



## Functions

In [4]:
#### Connect to FPL API
def get(url):
    response = requests.get(url)
    return json.loads(response.content)

## My Team

In [5]:
#### Get my fantasy team lineup
now = datetime.now()
my_team_id = '1341730'
current_gw = all_events[all_events['deadline_time'] < now]['gameweek_id'].max()

my_team_url = 'https://fantasy.premierleague.com/api/entry/{}/event/{}/picks/'.format(my_team_id,str(current_gw))

my_team = get(my_team_url)['picks']
my_team = pd.DataFrame(my_team)


In [6]:
all_players.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round',
       'player_code', 'cost_change_event', 'cost_change_event_fall',
       'cost_change_start', 'cost_change_start_fall', 'dreamteam_count',
       'player_type_id', 'ep_next', 'ep_this', 'event_points', 'form',
       'player_id', 'in_dreamteam', 'news', 'news_added', 'now_cost', 'photo',
       'PPG', 'selected_by_percent', 'status', 'team_id', 'team_code',
       'total_points', 'transfers_in_event', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'total_minutes',
       'goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
       'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
       'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity',
       'threat', 'ict_index', 'influence_rank', 'influence_rank_type',
       'creativity_rank', 'creativity_rank_type', 'threat_rank',
       'threat_rank_type', 'ict_index_rank', 'ict_index_rank_type',
       'corne

In [7]:
#all_players = all_players[['chance_of_playing_next_round', 'chance_of_playing_this_round', 'position_id', 'ep_next','ep_this', 'player_id', 'player_news', 'news_added', 'player_price', 'selected_by_percent', 'player_status','team_id', 'web_name']]

all_players.head()

,chance_of_playing_next_round,chance_of_playing_this_round,player_code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,player_type_id,ep_next,...,penalties_order,penalties_text,PP90,PPMM,VAPM,team_strength,team_short_name,strength_overall_home,strength_overall_away,position_name_short
0,0.0,0.0,37605,0,0,-2,2,0,3,0.0,...,NaN,NaN,NaN,0.000000,-0.294118,4,ARS,1240,1250,MID
1,100.0,100.0,54694,0,0,-3,3,0,3,2.0,...,1.0,NaN,3.333333,0.282051,0.111111,4,ARS,1240,1250,MID
2,NaN,NaN,84450,0,0,-1,1,0,3,1.5,...,NaN,NaN,2.211302,0.314815,-0.055556,4,ARS,1240,1250,MID
3,NaN,NaN,195735,0,0,-3,3,0,3,2.7,...,2.0,NaN,5.346535,0.259740,0.000000,4,ARS,1240,1250,MID
4,0.0,0.0,198849,0,0,-3,3,0,3,0.0,...,NaN,NaN,NaN,0.000000,-0.425532,4,ARS,1240,1250,MID


In [8]:
my_team

,element,position,multiplier,is_captain,is_vice_captain
0,363,1,1,False,False
1,255,2,1,False,False
2,232,3,1,False,False
3,470,4,1,False,False
4,259,5,1,False,False
5,390,6,1,False,True
6,254,7,2,True,False
7,37,8,1,False,False
8,377,9,1,False,False
9,460,10,1,False,False


In [9]:
my_team = my_team.merge(all_players, how = 'left', left_on="element", right_on="player_id")
my_team.drop(["element","position"], axis = 1, inplace = True)

In [10]:
my_team.sort_values(by=['player_type_id' , 'selected_by_percent'])['web_name']



11              Nyland
0             McCarthy
3               Kilman
1            Robertson
2             Chilwell
12               Saïss
4     Alexander-Arnold
14             Ritchie
13            Bissouma
7             Grealish
6                Salah
5                  Son
8                Adams
9              Jiménez
10              Werner
Name: web_name, dtype: object

In [11]:
my_team.to_csv (r'./data/' + season + '/Team_data/my_team.csv', index = False, header=True)

In [12]:
my_team

,multiplier,is_captain,is_vice_captain,chance_of_playing_next_round,chance_of_playing_this_round,player_code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,...,penalties_order,penalties_text,PP90,PPMM,VAPM,team_strength,team_short_name,strength_overall_home,strength_overall_away,position_name_short
0,1,False,False,NaN,NaN,58376,0,0,0,0,...,NaN,NaN,4.000000,0.888889,0.444444,3,SOU,1090,1120,GKP
1,1,False,False,NaN,NaN,122798,0,0,1,-1,...,NaN,NaN,4.500000,0.633803,0.352113,5,LIV,1330,1350,DEF
2,1,False,False,100.0,100.0,172850,1,-1,4,-4,...,NaN,NaN,9.000000,1.525424,1.186441,4,CHE,1250,1280,DEF
3,1,False,False,NaN,NaN,214048,0,0,2,-2,...,NaN,NaN,7.250000,1.714286,1.238095,4,WOL,1190,1210,DEF
4,1,False,False,NaN,NaN,169187,0,0,-1,1,...,NaN,NaN,2.843866,0.378378,0.108108,5,LIV,1330,1350,DEF
5,1,False,True,100.0,100.0,85971,0,0,5,-5,...,2.0,NaN,13.326180,1.210526,1.000000,4,TOT,1270,1290,MID
6,2,True,False,NaN,NaN,118748,0,0,4,-4,...,2.0,NaN,8.333333,0.669355,0.508065,5,LIV,1330,1350,MID
7,1,False,False,NaN,NaN,114283,0,0,3,-3,...,3.0,NaN,7.800000,1.068493,0.794521,3,AVL,1130,1150,MID
8,1,False,False,NaN,NaN,200439,0,0,-2,2,...,NaN,NaN,5.272727,0.827586,0.482759,3,SOU,1090,1120,FWD
9,1,False,False,NaN,NaN,102057,0,0,0,0,...,1.0,NaN,5.000000,0.588235,0.352941,4,WOL,1190,1210,FWD
